Scraping a wikipedia table https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [5]:
def scrap(url):
    page = requests.get(url) # get request to the url
    statusCode = page.status_code
    
    if statusCode == 200: 
        soup = BeautifulSoup(page.content, 'html.parser') # import raw html into BeautifulSoup
        
        wiki_table = soup.find('table', class_='wikitable sortable') # find table in the class
        
        # create 6 empty lists that will be the 6 columns in df
        symbol_list = [] 
        name_list = []
        sector_list = []
        subsector_list = []
        headquarter_list = []
        added_list = []
        
        count = 0
        
        for row in wiki_table.findAll('tr'): # iterate through each row in table
            cells = row.findAll('td')
            
            if len(cells) == 8: # check if there are 8 columns in the table
                symbol_list.append(cells[0].text.strip())
                name_list.append(cells[1].text.strip())
                sector_list.append(cells[2].text.strip())
                subsector_list.append(cells[3].text.strip())
                headquarter_list.append(cells[4].text.strip())
                added_list.append(cells[5].text.strip())
                
                count+=1 # put this inside if statement to ensure counter is only
                        # incremented for a valid row
            
            if count == 50:
                break

    # a dictionary; key is the column name; value is the list                 
    data = {
        'Ticker': symbol_list,
        'Name': name_list,
        'GICS Sector': sector_list,
        'GICS Sub-Industry': subsector_list,
        'Headquarters': headquarter_list,
        'Date Added': added_list
    }
    
    df = pd.DataFrame(data) # create a df using the dictionary
    return df, symbol_list # returns both dataframe and list of tickers
            
            

In [7]:
first_df, tickers = scrap('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_df

,Ticker,Name,GICS Sector,GICS Sub-Industry,Headquarters,Date Added
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06
5,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05
6,AMD,Advanced Micro Devices,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20
7,AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02
8,AFL,Aflac,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28
9,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",2000-06-05


Tiingo API

In [9]:
second_df = pd.DataFrame() # create empty dataframe & data will get appended later on

for ticker in tickers:
    
    base_url = 'https://api.tiingo.com/tiingo/daily/'
    f = 'snapshot/locale/us/markets/stocks/tickers/'
    token = '4cd0c549f40db38a6bc8ee76da516e1db0e19440'
    date = '2023-12-01'
    
    headers = {  # from documentation
        'Content-Type': 'application/json'
    }
    
    # making the url 
    url = f'{base_url}{ticker}/prices?startDate={date}&token={token}'
    print(url)

    # making the request
    r = requests.get(url, headers=headers)

    # Checking if the request was successful, only then will we attempt to process the JSON code
    if r.status_code == 200:

        data = r.json()
        
        latest_data = data[-1]  # the latest date is at the end of list

        stock_data = { # a dict with stock data
            'Stock Symbol': ticker,
            'Open': float(latest_data['open']),
            'High': float(latest_data['high']),
            'Low': float(latest_data['low']),
            'Close': float(latest_data['close']),
            'Volume': int(latest_data['volume'])
        }

        # append each row (company) at a time
        second_df = second_df.append(stock_data, ignore_index=True)
        
        print(f'{ticker} open price is {stock_data["Open"]}')
        print(f'{ticker} high price is {stock_data["High"]}')
        print(f'{ticker} low price is {stock_data["Low"]}')
        print(f'{ticker} close price is {stock_data["Close"]}')
        print(f'{ticker} volume is {stock_data["Volume"]}')
        print()
       
    else:
        print("API request was unsuccessful.")
        

https://api.tiingo.com/tiingo/daily/MMM/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
MMM open price is 104.98
MMM high price is 107.63
MMM low price is 104.96
MMM close price is 107.04
MMM volume is 5764490

https://api.tiingo.com/tiingo/daily/AOS/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
AOS open price is 79.85
AOS high price is 80.84
AOS low price is 79.65
AOS close price is 80.59
AOS volume is 1450210

https://api.tiingo.com/tiingo/daily/ABT/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
ABT open price is 107.83
ABT high price is 108.665
ABT low price is 106.68
ABT close price is 108.6
ABT volume is 6611835

https://api.tiingo.com/tiingo/daily/ABBV/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
ABBV open price is 154.36
ABBV high price is 155.16
ABBV low price is 152.7593
ABBV close price is 154.88
ABBV volume is 7869640

https://api.tiingo.com/tiingo/daily/ACN/pric

AMGN open price is 281.32
AMGN high price is 282.73
AMGN low price is 274.08
AMGN close price is 276.32
AMGN volume is 2935224

https://api.tiingo.com/tiingo/daily/APH/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
APH open price is 96.93
APH high price is 98.87
APH low price is 96.685
APH close price is 98.81
APH volume is 4056704

https://api.tiingo.com/tiingo/daily/ADI/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
ADI open price is 193.19
ADI high price is 200.26
ADI low price is 193.19
ADI close price is 200.17
ADI volume is 5610167

https://api.tiingo.com/tiingo/daily/ANSS/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
ANSS open price is 299.99
ANSS high price is 303.29
ANSS low price is 293.4223
ANSS close price is 294.9
ANSS volume is 864708

https://api.tiingo.com/tiingo/daily/AON/prices?startDate=2023-12-01&token=4cd0c549f40db38a6bc8ee76da516e1db0e19440
AON open price is 335.33
AON high price 

In [11]:
second_df 

,Stock Symbol,Open,High,Low,Close,Volume
0,MMM,104.98,107.6300,104.9600,107.04,5764490.0
1,AOS,79.85,80.8400,79.6500,80.59,1450210.0
2,ABT,107.83,108.6650,106.6800,108.60,6611835.0
3,ABBV,154.36,155.1600,152.7593,154.88,7869640.0
4,ACN,342.74,345.3900,340.7400,342.73,2551254.0
5,ADBE,593.00,593.1600,578.3000,584.64,11315347.0
6,AMD,138.89,141.8200,135.7250,138.00,76115174.0
7,AES,19.75,20.2400,19.4500,19.63,10522630.0
8,AFL,84.07,84.1700,80.7100,81.28,3841619.0
9,A,135.00,138.1700,135.0000,137.96,2950536.0


In [12]:
# Inner join on 'Ticker' from fir_df and Stock Symbol from second_df

merged_df = pd.merge(first_df, second_df, left_on='Ticker', right_on='Stock Symbol', how='inner')
merged_df

,Ticker,Name,GICS Sector,GICS Sub-Industry,Headquarters,Date Added,Stock Symbol,Open,High,Low,Close,Volume
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,MMM,104.98,107.6300,104.9600,107.04,5764490.0
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,AOS,79.85,80.8400,79.6500,80.59,1450210.0
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,ABT,107.83,108.6650,106.6800,108.60,6611835.0
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,ABBV,154.36,155.1600,152.7593,154.88,7869640.0
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,ACN,342.74,345.3900,340.7400,342.73,2551254.0
5,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05,ADBE,593.00,593.1600,578.3000,584.64,11315347.0
6,AMD,Advanced Micro Devices,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,AMD,138.89,141.8200,135.7250,138.00,76115174.0
7,AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,AES,19.75,20.2400,19.4500,19.63,10522630.0
8,AFL,Aflac,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28,AFL,84.07,84.1700,80.7100,81.28,3841619.0
9,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",2000-06-05,A,135.00,138.1700,135.0000,137.96,2950536.0


In [13]:
merged_df.describe()

,Open,High,Low,Close,Volume
count,50.000000,50.000000,50.000000,50.000000,5.000000e+01
mean,151.120200,153.507218,148.355032,150.596600,1.134009e+07
std,108.528086,109.367349,106.048257,107.324303,1.796356e+07
min,9.810000,9.977000,9.690000,9.710000,6.213800e+05
25%,82.122500,82.610000,79.915000,80.762500,2.509246e+06
50%,134.885000,136.437500,132.405000,134.470000,4.302432e+06
75%,188.827500,194.690000,188.792500,193.297500,7.826786e+06
max,593.000000,593.160000,578.300000,584.640000,7.611517e+07


In [14]:
merged_df.to_csv('merged_stock.csv', index = False)